In [1]:
%pylab inline
rcParams["figure.figsize"] = (16,5)

import sys
sys.path.insert(0, "..")

Populating the interactive namespace from numpy and matplotlib


In [2]:
!pip3 install pysptk

     |████████████████████████████████| 419 kB 7.2 MB/s 
  Created wheel for pysptk: filename=pysptk-0.1.18-cp37-cp37m-linux_x86_64.whl size=952269 sha256=5729e4ab82fec14b2537a17aad46d72d9728605903284c6e63adffb16b2eb5d2
  Stored in directory: /root/.cache/pip/wheels/44/95/38/580fe995213257289c69bf74f4d122e39cd134c731fe17a5a6
Successfully built pysptk


In [3]:
!pip3 install pyworld

     |████████████████████████████████| 212 kB 6.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyworld: filename=pyworld-0.3.0-cp37-cp37m-linux_x86_64.whl size=609908 sha256=0d40a868db022d37666e1bc95632f552767ad5bd188f3eaadad0513424228f10
  Stored in directory: /root/.cache/pip/wheels/e7/7c/11/c775fffa0e1e7b05a6604b4323408a77f80fb4ab304d96b5c6
Successfully built pyworld


In [4]:
from os.path import join, basename

In [5]:
import torch

from scipy.io import wavfile
import pysptk
from pysptk.synthesis import Synthesizer, MLSADF
import pyworld
from os.path import join, basename

#from nnmnkwii import preprocessing as P
#from nnmnkwii.paramgen import unit_variance_mlpg_matrix

#import gantts
#from hparams import vc as hp

import librosa
import librosa.display
import IPython
from IPython.display import Audio

In [ ]:
import os
 
# This is my path
path1="/content/data/cycle/"
name= "1.wav"
# to store files in a list
list_cycle = []
 
# dirs=directories
for (root, dirs, file) in os.walk(path1):
    for f in file:
        print(f)
        a=join(path1, name)
        list_cycle.append(a)

In [ ]:
path="/content/data/algan_vc/"
name= "1.wav"
# to store files in a list
list_algan = []
 
# dirs=directories
for (root, dirs, file) in os.walk(path):
    for f in file:
        print(f)
        a=join(path, name)
        list_algan.append(a)

In [ ]:
from tqdm import tnrange
fs = 16000
frame_period=5.0
hop_length = int(fs * (frame_period * 0.001))
ms_fftlen = 4096
modfs = fs / hop_length
ms_freq = np.arange(ms_fftlen//2 + 1)/ms_fftlen * modfs
ms_freq

In [ ]:
def compute_static_features(path):
    fs, x = wavfile.read(path)
    x = x.astype(np.float64)
    f0, timeaxis = pyworld.dio(x, fs, frame_period=5.0)
    f0 = pyworld.stonemask(x, f0, timeaxis, fs)
    spectrogram = pyworld.cheaptrick(x, f0, timeaxis, fs)
    aperiodicity = pyworld.d4c(x, f0, timeaxis, fs)
    alpha = pysptk.util.mcepalpha(fs)
    mc = pysptk.sp2mc(spectrogram, order=24, alpha=alpha)
    c0, mc = mc[:, 0], mc[:, 1:]
    return mc

In [ ]:
import numpy as np




def modspec(x, n=4096, norm=None, return_phase=False):
    
    # DFT against time axis
    s_complex = np.fft.rfft(x, n=n, axis=0, norm=norm)
    assert s_complex.shape[0] == n // 2 + 1
    R, im = s_complex.real, s_complex.imag
    ms = R * R + im * im

    # TODO: this is ugly...
    if return_phase:
        return ms, np.exp(1.0j * np.angle(s_complex))
    else:
        return ms

In [ ]:
def mean_modspec(files):
    mss = []
    for idx in tnrange(len(files)):
        f = files[idx] 
        mgc = compute_static_features(f)
        #print(mgc)
        #b=log(modspec(mgc, n=ms_fftlen))
        #print(b)
        #print(np.any(b<=0))

    
        #print(b)
        ms = np.log(modspec(mgc, n=ms_fftlen))
        mss.append(ms)
    return np.mean(np.array(mss), axis=(0,))

In [ ]:
ms_in2out = mean_modspec(list_algan)
ms_in2out

In [ ]:
import librosa

In [ ]:
ms_in2out = mean_modspec(list_algan)
ms_in2out_new = mean_modspec(list_cycle)

In [ ]:
#MSD difference



new=0
for i in range(24):
    a=ms_in2out[i, :].T
    b=ms_in2out_new [i,:].T
    diff=np.mean(np.absolute(a-b))
    diff=(np.inner(diff, diff))
    new=new+diff


MSD=math.sqrt(1/len(mean_modspec(list_algan).T))*math.sqrt(new)

#a=ms_in2out[:, :].T
#b=ms_in2out_new [:,:].T
#diff=a-b
#diff=np.absolute(np.mean(diff))
#math.sqrt(np.inner(diff, diff))